Bevor Sie dieses Problem einreichen, stellen Sie sicher, dass alles wie erwartet läuft. Starten Sie zuerst **den Kernel neu** (wählen Sie in der Menüleiste Kernel$\rightarrow$Restart) und führen Sie dann **jede Zelle aus** (wählen Sie in der Menüleiste Cell$\rightarrow$Run Cells ausführen). Wenn Sie Fragen zum Jupyterhub oder zur Aufgabe haben, stehe ich Ihnen gerne zur Verfügung: quang.tran@fh-dortmund.de

Stellen Sie sicher, dass Sie alle Stellen ausfüllen, an denen "YOUR CODE HERE" oder "YOUR ANSWER HERE" steht, sowie Ihren Namen und Ihre Mitarbeiter unten:


In [ ]:
NAME = ""
Mitarbeiter = ""

---

<img src="fhlogo.png" width="350">

# Kartierung bei bekannten Posen (6.2 von Hertzberg et al.)

Bevor wir uns der allgemeinen Lösung des SLAM-Problems zuwenden, betrachten wir als Spezialfall Erstellen einer Karte unter der Bedingung, dass die Roboterposen genau bekannt sind (Localisierung schon bekannt). Dieses Tutorial zeigt, wie man LIDAR-Messungen (Entfernungsmessungen) aus einer Datei einliest und in ein Belegungsraster konvertiert.

## Teil A: Theoretisch Belegtheitskarte/Occupancy Grid Map( Punkte)
Bitte füllen Sie alle untenstehenden Antworten aus:

Frage 1

YOUR ANSWER HERE

Frage 2

YOUR ANSWER HERE

## Teil B: Algorithmus Belegtheitskarte/Occupancy Grid Map( points)

Belegtheitskarte [Hans Moravec, A.E. Elfes: High resolution maps from wide angle sonar, Proc. IEEE Int. Conf. Robotics Autom. (1985)] sind ein beliebter, probabilistischer Ansatz zur Darstellung der Umgebung. Das Gitter ist im Grunde eine diskrete Darstellung der Umgebung, die anzeigt, ob eine Gitterzelle besetzt ist oder nicht. Hier wird die Karte als Numpy-Array dargestellt, und Zahlen nahe bei 1 bedeuten, dass die Zelle besetzt ist (schwarz markiert), Zahlen nahe bei 0 bedeuten, dass sie frei ist (hellgrau markiert). Das Raster hat die Möglichkeit, unbekannte (unbeobachtete, schwarz markiert) ereiche darzustellen, die nahe bei 0,5 liegen.

Eingabe : Sequenz von LIDAR Messwerten $z_t$ und korrekten Posen $x_t$.

Ausgabe:  Belegtheitsw.keit Bel($m_{x,z}$) für jede Rasterzelle (x, z).


In [ ]:
import scipy.io
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

class Map():
    def __init__(self, xsize, ysize, grid_size):
        self.xsize = xsize+2 # Add extra cells for the borders
        self.ysize = ysize+2
        self.grid_size = grid_size # save this off for future use
        self.log_prob_map = np.zeros((self.xsize, self.ysize)) # set all to zero

        self.alpha = 1.0 # The assumed thickness of obstacles
        self.beta = 5.0*np.pi/180.0 # The assumed width of the laser beam
        self.z_max = 150.0 # The max reading from the laser

        # Pre-allocate the x and y positions of all grid positions into a 3D tensor
        # (pre-allocation = faster)
        self.grid_position_m = np.array([np.tile(np.arange(0, self.xsize*self.grid_size, self.grid_size)[:,None], (1, self.ysize)),
                                         np.tile(np.arange(0, self.ysize*self.grid_size, self.grid_size)[:,None].T, (self.xsize, 1))])

        # Log-Probabilities to add or remove from the map 
        self.l_occ = np.log(0.65/0.35)
        self.l_free = np.log(0.35/0.65)

    def update_map(self, pose, z):

        dx = self.grid_position_m.copy() # A tensor of coordinates of all cells
        dx[0, :, :] -= pose[0] # A matrix of all the x coordinates of the cell
        dx[1, :, :] -= pose[1] # A matrix of all the y coordinates of the cell
        theta_to_grid = np.arctan2(dx[1, :, :], dx[0, :, :]) - pose[2] # matrix of all bearings from robot to cell

        # Wrap to +pi / - pi
        theta_to_grid[theta_to_grid > np.pi] -= 2. * np.pi
        theta_to_grid[theta_to_grid < -np.pi] += 2. * np.pi

        dist_to_grid = scipy.linalg.norm(dx, axis=0) # matrix of L2 distance to all cells from robot

        # For each laser beam
        for z_i in z:
            r = z_i[0] # range measured
            b = z_i[1] # bearing measured

            # Calculate which cells are measured free or occupied, so we know which cells to update
            # Doing it this way is like a billion times faster than looping through each cell (because vectorized numpy is the only way to numpy)
            free_mask = (np.abs(theta_to_grid - b) <= self.beta/2.0) & (dist_to_grid < (r - self.alpha/2.0))
            occ_mask = (np.abs(theta_to_grid - b) <= self.beta/2.0) & (np.abs(dist_to_grid - r) <= self.alpha/2.0)

            # Adjust the cells appropriately
            self.log_prob_map[occ_mask] += self.l_occ
            self.log_prob_map[free_mask] += self.l_free

if __name__ == '__main__':

    # load matlab generated data (located at http://jamessjackson.com/files/index.php/s/sdKzy9nnqaVlKUe)
    data = scipy.io.loadmat('state_meas_data.mat')
    state = data['X']
    meas = data['z']

    # Define the parameters for the map.  (This is a 100x100m map with grid size 1x1m)
    grid_size = 1.0
    map = Map(int(100/grid_size), int(100/grid_size), grid_size)

    plt.ion() # enable real-time plotting
    plt.figure(1) # create a plot
    for i in tqdm(range(len(state.T))):
        map.update_map(state[:,i], meas[:,:,i].T) # update the map

        # Real-Time Plotting 
        # (comment out these next lines to make it run super fast, matplotlib is painfully slow)
        #plt.clf()
        pose = state[:,i]
        circle = plt.Circle((pose[1], pose[0]), radius=3.0, fc='y')
        plt.gca().add_patch(circle)
        arrow = pose[0:2] + np.array([3.5, 0]).dot(np.array([[np.cos(pose[2]), np.sin(pose[2])], [-np.sin(pose[2]), np.cos(pose[2])]]))
        plt.plot([pose[1], arrow[1]], [pose[0], arrow[0]])
        plt.imshow(1.0 - 1./(1.+np.exp(map.log_prob_map)), 'Greys')
        plt.pause(0.005)

    # Final Plotting
    plt.ioff()
    plt.clf()
    plt.imshow(1.0 - 1./(1.+np.exp(map.log_prob_map)), 'Greys') # This is probability
    plt.imshow(map.log_prob_map, 'Greys') # log probabilities (looks really cool)
    plt.show()

Aus den Abständen und den Winkeln lassen sich leicht die x- und y-Koordinaten mit sin und cos bestimmen. Für die Darstellung wird matplotlib.pyplot (plt) verwendet.

Die Datei lidar_to_grid_map.py enthält praktische Funktionen, mit denen eine 2D-Entfernungsmessung in eine Rasterkarte umgewandelt werden kann. Zum Beispiel gibt die bresenham die eine gerade Linie zwischen zwei Punkten in eine Rasterkarte. Schauen wir uns an, wie das funktioniert:

Um leere Bereiche zu füllen, kann ein warteschlangenbasierter Algorithmus verwendet werden, der auf eine initialisierte Belegungskarte angewendet werden kann. Der Mittelpunkt ist gegeben: Der Algorithmus (lg.flood_fill) prüft in jeder Iteration auf Nachbarelemente und stoppt die Expansion bei Hindernissen und freien Grenzen.

 Lassen Sie uns diese Bresenham auf reale Daten anwenden:

Gratulation, Sie haben gerade eine Belegtheitskarte vom Laserscanner erstellt 